<a href="https://colab.research.google.com/github/NicholasBaraghini/Ball-and-Beam-system-Optimal-Control/blob/main/OPTCON_Grp21_Ball_and_Beam_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**OPTIMAL CONTROL EXAM PROJECT**

 Group 21:
   - Baraghini Nicholas 0000984044
   - Curto Fabio
   - Iadarola Federico

In [2]:
# Libraries
import numpy as np
import matplotlib

**SYSTEM DYNAMICS DYNAMICS** (Ball and Beam)

In [ ]:
# Ball and Beam Dynamics 

def BB_Dynamics(x_sys, u_in, p_tens, params)
  # inputs:
  #   - x_sys : system state at current time t 
  #   - u_in  : input at current time t
  #   - p_tens: tensor product term
  #   - params: list of parameters

  # Prarameter Extraction
  gg = params(1);
  mm


  # outputs: (the fucnction returns an output structure with the follows entries)
  #   - x_next : system state at state (t+1)
  #   - fx     : gradient of the system dynamics w.r.t. the system state at time t
  #   - fu     : gradient of the system dynamics w.r.t. the input at time t
  #   - pfxx   : tensor product
  #   - pfux   : tensor product

